In [50]:
import matplotlib.pyplot as plt

import networkx as nx

import numpy as np

# For reading files
import os

# Data manipulation
import pandas as pd

# Text cleaning
import re

import requests

## Text processing
# NLP
import spacy
from spacy.lang.en import English
from spacy.matcher import Matcher
from spacy.tokens import Span
nlp = spacy.load("en_core_web_sm")

# Sentence boundary detection
import pysbd

# More NLP for fun
from textblob import TextBlob

from tqdm import tqdm

# Pipeline creation
from sklearn.pipeline import Pipeline

# Text extraction
from tika import parser

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [51]:
# Wanted to see details
pd.options.display.max_rows = 999

In [8]:
read_file = pd.read_excel (r'C:\Users\sambe\Projects\Cover_Letter_Analysis\data\Data_tobecompiled\Accounts_Payable_Specialist_Chicago_IL_2020-06-08\Accounts_Payable_Specialist_Chicago_IL_2020-06-08 .xlsx')
read_file.to_csv (r'accounts_payable_specialist.csv', index = None, header=True)

In [11]:
cover_letter = pd.read_csv('accounts_payable_specialist.csv')

In [12]:
cover_letter

,Accounts_Payable_Specialist_Chicago_IL_2020-06-08,Unnamed: 1
0,NaN,NaN
1,Jun 7,NaN
2,Cassandra Clingon,"Cassandra Clingon \nChicago, IL (309) 838-7117..."
3,Jasmine Hill,"Dear Recipient: Jasmine Hill\nBUSINESS, MODELI..."
4,Angela Struckhoff,"\nDear Hiring Team,\n\nAs a highly skilled Tea..."
...,...,...
142,Alexandria Dotdot,To Whom It May Concern:\n\nMy name is Alexandr...
143,Elizabeth Shea-Brown,To Whom It Might Concern:\nI was excited to se...
144,Dalibor Mihajlovski,DALIBOR MIHAJLOVSKI\n135 A Grand Canyon Dr. Ap...
145,Elizabeth F. Gundlach,"May 30, 2020\n\nRookieplay\nAttn: Human Resour..."


In [18]:
cover_letter_df = cover_letter.drop([0,1], axis=0)

In [19]:
cover_letter_df.columns = ['username', 'raw_cover_letters']

In [27]:
cover_letter_df = cover_letter_df.reset_index(drop=True)

In [38]:
cover_letter_df = cover_letter_df.sort_values('username').reset_index(drop=True)

In [37]:
cover_letter_df.username[0] = 'Jessica Santiago'

In [40]:
cover_letter_df.to_csv('accounts_payable_specialist.csv')

In [47]:
cover_letter_df = cover_letter_df.drop(['title'], axis=1)


In [48]:
cover_letter_df.insert(1, 'title', 'Accounts Payable Specialist')


In [49]:
cover_letter_df

,username,title,raw_cover_letters
0,Adrian Mitchell,Accounts Payable Specialist,To whom it may concern --\n\nMy name is Adrian...
1,Adriana Jauregui,Accounts Payable Specialist,I am interested in applying for the position a...
2,Adriana Osman,Accounts Payable Specialist,"June 1, 2020\nDear Hiring Manager,\n\nI am con..."
3,Agron Podrimqaku,Accounts Payable Specialist,"Dear Sir/Madam,\nThis letter is to express my ..."
4,Alberto Lopez,Accounts Payable Specialist,Alberto Lopez\n915-873-2575\nalberto.Lopez15@y...
...,...,...,...
140,Wayne Louie,Accounts Payable Specialist,To the hiring manager of Rookieplay:\nIn my ei...
141,Wendy Taylor,Accounts Payable Specialist,WENDY V. TAYLOR\n214 N. Aldine Ave. • Park Rid...
142,Yanyan Chen,Accounts Payable Specialist,"Yanyan Chen\n417 W Cermak RD,\nApt 1B,\nChicag..."
143,Yuliia Tyszka,Accounts Payable Specialist,My resume will tell you I am a senior finance ...


In [135]:
# Construct dataframe
resumes_df = pd.DataFrame(columns=['username', 'raw_resume'])
resumes_df

,username,raw_resume


In [142]:
all_text = []
filenames = []

def extract_data():
    for filename in os.listdir('data/Data_tobecompiled/Accounts_Payable_Specialist_Chicago_IL_2020-06-08/'):
        parsed = parser.from_file('data/Data_tobecompiled/Accounts_Payable_Specialist_Chicago_IL_2020-06-08/' + filename)
        text = parsed['content']
        if parsed['content'] == None:
            print(filename + " is bad.")
        filenames.append(filename)
#         resumes_df.username = filenames
        try:
            text = text.replace('\n', '')          
        except:
            pass
        all_text.append(text)
#         print("extracting data from " + filename)
    resumes_df.raw_resume = all_text
    resumes_df.username = filenames
#     resumes_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
    
extract_data()

In [143]:
len(all_text)

626

In [144]:
len(filenames)

626

In [145]:
resumes_df

,username,raw_resume
0,Accounts_Payable_Specialist_Chicago_IL_2020-06-08 .xlsx,"Sheet 1\tAccounts_Payable_Specialist_Chicago_IL_2020-06-08 \tJun 7\tCassandra Clingon \tCassandra Clingon Chicago, IL (309) 838-7117 cassandraclingon@yahoo.com 3rd June 2020 RookiePlay Los Angeles..."
1,Erika's Resume.docx,"Erika Lester Erika Lester\t\t\t 921 E.40th StChicago, IL, 60653elester89@gmail.com ..."
2,ResumeAdamO'Leary.pdf,"Indeed ResumeAdam O'LearyBartender -Cindy's RooftopChicago, IL 60647adamoleary3_95q@indeedemail.com813-808-1900BASSET and Servsafe safe certified Barback with serving and foodrunning experience wo..."
3,ResumeAdrianaJauregui.pdf,"Indeed ResumeAdriana JaureguiArea ManagerLas Vegas, NV 89119adrianajauregui3_th9@indeedemail.com(702) 986-2747Ten years of experience in fulfilling various administrative and customer service func..."
4,ResumeAdrianaOsman.docx,"Adriana J. Osman1459 Turkey Trl, Inverness, ILL 60067adrij76@hotmail.com * (515)689-1377_____________________________________________________________________________Education: Nova Southeastern Un..."
5,ResumeAdrianMitchell.pdf,"Indeed ResumeAdrian MitchellMalden, MA 02148adrianmitchell748_y5a@indeedemail.com(774) 254-5474Authorized to work in the US for any employerWork ExperienceOffice AssistantFrom The Top - Boston, MA..."
6,ResumeAgronPodrimqaku.docx,"Agron Podrimqaku Str Nena Tereze. Linda floor 5 No# 39, Kosovo 12010,+38344177819 agron_p@hotmail.com\t Dardania,28 Nentori SU 1/3 Hy2-2, Pristhine, Kosovo 10000 +38649177819 agron_p@hotmail.com..."
7,ResumeAimaKarar.pdf,"Indeed ResumeAima KararCashier / Customer Service Representative - Target CorporationChicago, IL 60645aimakarar7_3qv@indeedemail.com(773) 467-6867#readytoworkAuthorized to work in the US for any e..."
8,ResumeAlbertoLopez.pdf,"Angel Lopez, Jr ALBERTO LOPEZ, MBA El Paso, Texas 79936 (915) 873 – 2575 Alberto.Lopez15@yahoo.com PROFESSIONAL PROFILE Accredited bi-lingual business professional with extensive knowledge ..."
9,ResumeAleciaSims.pdf,"Indeed ResumeAlecia SimsOak Forest, IL 60452aleciasims2_5cw@indeedemail.com(708) 536-0914Work ExperienceAccounting/Finance AnalystL&W Supply - Chicago, IL2019 to Present• Use Woodstock to run and ..."
